https://docs.microsoft.com/fr-fr/learn/modules/register-and-deploy-model-with-amls/2-deploy-model

In [ ]:
!pip install azureml-sdk
!pip install azureml
!pip install azureml-core
!pip install azureml-contrib-services
!pip install azureml-contrib-functions

In [ ]:
from azureml.core import Workspace

ws = Workspace.create(name='myworkspace01',
               subscription_id='bf72f9c5-c559-47bb-a514-e1c1ebcd03b7',
               resource_group='P9ressource',
               create_resource_group=True,
               location='westeurope'
               )

Deploying KeyVault with name myworkspkeyvault4e5fdc69.
Deploying StorageAccount with name myworkspstorage2654f4a08.
Deploying AppInsights with name myworkspinsights0b4bb542.
Deployed AppInsights with name myworkspinsights0b4bb542. Took 6.6 seconds.
Deployed KeyVault with name myworkspkeyvault4e5fdc69. Took 24.83 seconds.
Deployed StorageAccount with name myworkspstorage2654f4a08. Took 29.1 seconds.
Deploying Workspace with name myworkspace01.
Deployed Workspace with name myworkspace01. Took 26.6 seconds.


In [ ]:
from azureml.core import Workspace

ws = Workspace.get(name='myworkspace01',
               subscription_id='bf72f9c5-c559-47bb-a514-e1c1ebcd03b7',
               resource_group='P9ressource')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F2K9STT3R to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [ ]:
ws.delete()

In [ ]:
ws.get_default_compute_target

<bound method Workspace.get_default_compute_target of Workspace.create(name='myworkspace01', subscription_id='bf72f9c5-c559-47bb-a514-e1c1ebcd03b7', resource_group='P9ressource')>

# 2 models :


1.   Collaborative Filtering
2.   Content-Based Filtering



# Inscrire un modèle à partir d’un fichier local

In [ ]:
import os
import urllib.request
from azureml.core.model import Model
# Download model
url_collaborative_Filtering = '/content/drive/MyDrive/Colab Notebooks/P9/model/Collaborative_Filtering.joblib'
url_sparse_user_item = '/content/drive/MyDrive/Colab Notebooks/P9/model/sparse_user_item.joblib'

# Register model
model_col = Model.register(workspace = ws,
                        model_path =url_collaborative_Filtering,
                        model_name = "model_col",
                        description = "Collaboratif filtering")



Registering model model_col


In [ ]:
# Register model
sparse_user_item  = Model.register(workspace = ws,
                        model_path =url_sparse_user_item,
                        model_name = "user_item",
                        description = "Matrice user_item",)

Registering model user_item


In [ ]:
model_col.get_model_path(model_name = "model_col")

In [ ]:
import os
import urllib.request
from azureml.core.model import Model
# Download model
url_ContentBased_Filtering = '/content/drive/MyDrive/Colab Notebooks/P9/model/articles_embeddings.pickle'
url_clicks = '/content/drive/MyDrive/Colab Notebooks/P9/data/clicks_small.csv'

# Register model
model_contbase = Model.register(workspace = ws,
                        model_path =url_ContentBased_Filtering,
                        model_name = "model_contbase",
                        description = "ContentBased_Filtering")



Registering model model_contbase


In [ ]:
# Register model
clicks  = Model.register(workspace = ws,
                        model_path =url_clicks,
                        model_name = "clicks",
                        description = "cliks",)

Registering model clicks


# Script d'entrée 1er Model

# Créer l'environnement

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("implicit")
myenv.add_conda_package("pickle5")
myenv.add_conda_package("numpy")


# Save the environment config as a .yml file
env_file = '/content/drive/MyDrive/Colab Notebooks/P9/Azure/service_files/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

Saved dependency info in /content/drive/MyDrive/Colab Notebooks/P9/Azure/service_files/env.yml


# Définir une configuration d’inférence

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python",
                                              source_directory = '/content/drive/MyDrive/Colab Notebooks/P9/Azure/service_files',
                                              entry_script="script.py",
                                              conda_file="env.yml")

## ACI



In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

model_col = ws.models['model_col']
sparse_user_item = ws.models['user_item']
aci_service_name = "acirec"

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(ws, aci_service_name, [model_col, sparse_user_item], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)

print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-22 15:51:51+00:00 Creating Container Registry if not exists..
2021-04-22 15:52:12+00:00 Registering the environment.
2021-04-22 15:52:13+00:00 Building image..
2021-04-22 15:59:44+00:00 Generating deployment configuration.
2021-04-22 15:59:46+00:00 Submitting deployment to compute..
2021-04-22 15:59:55+00:00 Checking the status of deployment acirec..
2021-04-22 16:03:17+00:00 Checking the status of inference endpoint acirec.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


# Script d'entrée 2eme Model

# Créer l'environnement, Définir une configuration d’inférence, ACI et test API

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python",
                                              source_directory = '/content/drive/MyDrive/Colab Notebooks/P9/Azure/service_files',
                                              entry_script="s4.py",
                                              conda_file="env_2.yml")
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

model_contbase = ws.models['model_contbase']
clicks = ws.models['clicks']

aci_service_name = "acicontbase2gb2mod"

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2)

service = Model.deploy(ws, aci_service_name, [model_contbase, clicks], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)

print(service.state)
import requests
import json

uri = 'http://61ba19cf-db24-403e-82bf-88dde2a9adcc.westeurope.azurecontainer.io/score'

data = {"articleId":"251"}
input_data = json.dumps(data) 

headers = {'Content-Type': 'application/json'}

response = requests.post(uri, data=input_data, headers=headers)
print('myapi:1 :response OK' , response)

response.json()

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-29 14:41:02+00:00 Creating Container Registry if not exists.
2021-04-29 14:41:02+00:00 Registering the environment.
2021-04-29 14:41:03+00:00 Use the existing image.
2021-04-29 14:41:03+00:00 Generating deployment configuration.
2021-04-29 14:41:04+00:00 Submitting deployment to compute.
2021-04-29 14:41:08+00:00 Checking the status of deployment acicontbase2gb2mod..
2021-04-29 14:48:49+00:00 Checking the status of inference endpoint acicontbase2gb2mod.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
myapi:1 :response OK <Response [200]>


['5418', '10055', '117575', '122032', '5393']